In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
from ddm.run_split import SplitMnistGenerator
from ddm.alg.cla_models_multihead import MFVI_IBP_NN
#from ddm.alg.vcl import run_vcl, run_vcl_ibp
from copy import deepcopy

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# IBP network

In [2]:
hidden_size = [256, 256]
batch_size = None
no_epochs = 100
single_head = False

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
batch_size=None
single_head=True
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
for task_id in range(data_gen.max_iter):
    x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=None, prev_log_variances=None)
    mf_model.train(x_train, y_train, head, no_epochs, bsize)
    mf_weights, mf_variances = mf_model.get_weights()

    # Incorporate coreset data and make prediction
    x_coresets = []
    y_coresets = []
    acc = get_scores(mf_model, x_testsets, y_testsets, x_coresets, y_coresets, hidden_size, no_epochs, single_head, batch_size)
    all_acc = concatenate_results(acc, all_acc)

    mf_model.close_session()

inputs: (?, 784)
act: (100, ?, 784)
beta a: (784, 256)
beta b: (784, 256)
slice: (100, 255)
logpis: (100, 784, 256)
beta a: (256,)
beta b: (256,)
slice: (100, 255)
logpis: (100, 1, 256)
beta a: (256, 256)
beta b: (256, 256)
slice: (100, 255)
logpis: (100, 256, 256)
beta a: (256,)
beta b: (256,)
slice: (100, 255)
logpis: (100, 1, 256)
Wtask_m :<unknown>
beta a: <unknown>
beta b: <unknown>
slice: <unknown>
logpis: <unknown>
beta a: <unknown>
beta b: <unknown>
slice: <unknown>
logpis: <unknown>
logpis_W: <unknown>
logpis_b: <unknown>
beta a: (784, 256)
beta b: (784, 256)
slice: (100, 255)
logpis: (100, 784, 256)
beta a: ()
beta b: ()
slice: (100, 255)
logpis: (100, 784, 256)
log_post: (100, 784, 256)
log_prior: (100, 784, 256)
log_sample: (100, 784, 256)
bern kl: ()
beta a: (256,)
beta b: (256,)
slice: (100, 255)
logpis: (100, 784, 256)
beta a: ()
beta b: ()
slice: (100, 255)
logpis: (100, 784, 256)
log_post: (100, 784, 256)
log_prior: (100, 784, 256)
log_sample: (100, 784, 256)
bern kl: 

InvalidArgumentError: Incompatible shapes: [10,12665,256,1] vs. [10,256,256]
	 [[{{node gradients/mul_253_grad/BroadcastGradientArgs}} = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/mul_253_grad/Shape, gradients/Mul_234_grad/Shape)]]

Caused by op 'gradients/mul_253_grad/BroadcastGradientArgs', defined at:
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/skessler/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", 

# Split MNIST

In [ ]:
hidden_size = [256, 256]
batch_size = None
no_epochs = 100
single_head = False

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
vcl_result = run_vcl_ibp(hidden_size, no_epochs, data_gen, batch_size, single_head) # runs loop of tasks, trains/re-trains vcl, evaluates vcl
print(vcl_result)

# # Run random coreset VCL
# tf.reset_default_graph()
# tf.set_random_seed(12)
# np.random.seed(1)
#
# coreset_size = 40
# data_gen = SplitMnistGenerator()
# rand_vcl_result = vcl.run_vcl(hidden_size, no_epochs, data_gen,
#     coreset.rand_from_batch, coreset_size, batch_size, single_head)
# print(rand_vcl_result)
#
# # Run k-center coreset VCL
# tf.reset_default_graph()
# tf.set_random_seed(12)
# np.random.seed(1)
#
# data_gen = SplitMnistGenerator()
# kcen_vcl_result = vcl.run_vcl(hidden_size, no_epochs, data_gen,
#     coreset.k_center, coreset_size, batch_size, single_head)
# print(kcen_vcl_result)

# Plot average accuracy
vcl_avg = np.nanmean(vcl_result, 1)
# rand_vcl_avg = np.nanmean(rand_vcl_result, 1)
# kcen_vcl_avg = np.nanmean(kcen_vcl_result, 1)
plt.plot(vcl_avg)
plt.show()

In [ ]:
vcl_avg